## Color Detection

In [7]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import scipy


In [84]:
# def G(x, y, sigma, img_height, img_width):



def gaussian_kernel(img_height, img_width, kernel_relative_size = 0.1, verbose=False):
    xs = scipy.signal.gaussian(img_width, std= img_width * kernel_relative_size)
    ys = scipy.signal.gaussian(img_height, std=img_height * kernel_relative_size)
    kernel = np.array([[x*y for x in xs] for y in ys])
    return kernel

In [90]:
import cv2
def readImg(filepath):
    img = cv2.imread(filepath)
    # img.resize()
    height = len(img)
    width= len(img[0])
    return img, height, width

In [97]:
FOLDER_PATH = 'E:\\img_test_input\\unlabeled\\'
FILEPATH = 'unlabeled-0001.jpg'
IMG_PATH = FOLDER_PATH + FILEPATH
print(IMG_PATH)
img, height, width = readImg(IMG_PATH)
g_kernel = gaussian_kernel(height, width,kernel_relative_size=0.01)


E:\img_test_input\unlabeled\img_00000034.jpg


In [109]:
def rgb_to_hex(r, g, b):
  return ('{:02X}' * 3).format(r, g, b)


In [111]:
img_colors = np.array(img.reshape((-1, 3)), dtype=float)
# img_colors = img_colors.dtype ='float'
g_kernel = g_kernel.reshape(-1)
# whitened_colors  = scipy.cluster.vq.whiten(img_colors)

bgr_total = [0, 0, 0]
for gauss_val, img_bgr_arr in zip(g_kernel, img_colors):
    bgr_total += img_bgr_arr * gauss_val
bgr_total = bgr_total / np.sum(g_kernel)
blue = int (bgr_total[0])
green = int (bgr_total[1])
red = int (bgr_total[2])

rgb_to_hex(red, green, blue)

# max_val = np.max(whitened_colors)

# k_means, distortion = scipy.cluster.vq.kmeans(whitened_colors, k_or_guess=5)
# adj_k_means = k_means * (256 // max_val)
# print(k_means)
# print(distortion)

'1D56C7'

In [ ]:
class KMeansReducedPalette:
    def __init__(self, num_colors):
        self.num_colors = num_colors
        # Random state for reproducibility.
        self.kmeans = KMeans(num_colors, random_state=0xfee1600d)
        self.source_pixels = None

    def _preprocess(self, image):
        assert image.shape[-1] == 3, 'image must have exactly 3 color channels'
        assert image.dtype == 'uint8', 'image must be in np.uint8 type'

        # Flatten pixels, if not already.
        if len(image.shape) > 2:
            return image.reshape(-1, 3)

        return image

    def fit(self, image):
        image_cpy = image.copy()
        self.source_pixels = self._preprocess(image_cpy)
        self.kmeans.fit(self.source_pixels)

    def recolor(self, image):
        original_shape = image.shape
        image = self._preprocess(image)
        recolor_idx = self.kmeans.predict(image)
        recolor = self.kmeans.cluster_centers_[recolor_idx]
        recolor = recolor.reshape(original_shape)

        return np.round(recolor).astype(np.uint8)